In [146]:

import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt


from datetime import datetime
from datetime import *
import os
import glob

In [2]:
import nltk
stops = nltk.corpus.stopwords.words('english')

stops.append('image')
stops.append('copyright')
stops.append('getty')
stops.append('images')
stops.append('caption')
stops.append('videothe')
stops.append('video')
stops.append('bbc_politics')
stops.append('reuters/getty')
stops.append('(cnn)')
stops.append('cnn')
stops.append('вЂ”')
stops.append('photos:')
stops.append('photos')
stops.append('file')
stops.append('photo:')
stops.append('photo')
stops.append('reuters')

print(stops)
print(len(stops))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [3]:
############ ЧАСОВОЙ
# Читаю часовой индекс
# Вычисляю разницу между следующим часом и текущим, и присваиваю эту дельту у текущему
# То есть в текущий час я знаю как измениться индекс в следующий час
sp500 = pd.read_csv(r"C:\Users\User\SANDP-500_NEW_Hour.csv", sep=',', header = 0, index_col =None, comment='#')


# sp500 = pd.read_csv(r"C:\Users\User\SANDP-500_30_MIN.csv", sep=',', header = 0, index_col =None, comment='#')
# sp500 = sp500.reset_index(drop='True')
sp500 = sp500.rename(columns={'<OPEN>': 'Open'})

for i in range(len(sp500) -1):
    sp500['Open'][i] = sp500['Open'][i+1] - sp500['Open'][i]    
    sp500 = sp500.rename(index = {i : datetime.strptime(str(sp500['<DATE>'][i]) + str(sp500['<TIME>'][i]), '%Y%m%d%H%M%S')} )
    

sp500 = sp500.drop(columns = ['<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<CLOSE>', '<DATE>'])

sp500 = sp500[:-1] # удаляю последний

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [149]:
print(len(sp500))
sp500.head()

872


,Open
2018-11-12 10:00:00,-15.43
2018-11-12 11:00:00,-11.91
2018-11-12 12:00:00,-4.06
2018-11-12 13:00:00,-3.34
2018-11-12 14:00:00,2.65


In [5]:
# Читаю все новости и объединяю в одну переменную типа DataFrame
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]
directory = r"C:\Users\User\data"
dir_list = get_immediate_subdirectories(directory)


for i in range(len(dir_list)):
    dir_list[i] = ''.join(   [r"C:\Users\User\data", "\\" , dir_list[i]]   )


news = pd.DataFrame([])
extension = 'csv'

news = news.astype('str')
for dir in dir_list:
    os.chdir(dir)
    #print(dir)
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    for names in all_filenames:
        if not (names == "WASHINGTONPOST_WORLD.csv" or names == "BREITBART.csv" or names == "BBC_WORLD.csv" or names == "YAHOO_WORLD.csv" or names == "REUTERS_BUSINESS.csv" or
               names == "CNN_LATEST.csv" or names == "REUTERS_WORLD.csv" or names == "NYT_WORLD.csv" or names == "REUTERS_COMPANY.csv" or names == "INTERFAX_ALL.csv"):
            path = dir + "\\" + names
            data = pd.read_csv(path, sep=';', header = None, index_col =None, comment='#')
            data.dropna(axis='index', how='any', thresh=None, subset=None, inplace=True)
            news = pd.concat([news, data], ignore_index=True)
        elif not names == "INTERFAX_ALL.csv":
            #print(names)
            path = dir + "\\" + names
            try:
                data = pd.read_csv(path, sep=';', header = None, index_col =None)
                data.dropna(axis='index', how='any', thresh=None, subset=None, inplace=True)
                news = pd.concat([news, data], ignore_index=True)
            except pd.errors.EmptyDataError:
                print('EmptyDataError')
            except pd.errors.ParserError:
                print('ParserError')
                
news = news.drop([0, 1, 4, 5], axis='columns')
news = news.rename(columns={2: 'headlines'})
news = news.rename(columns={3: 'Date'})


news.drop_duplicates(subset = 'headlines', keep = 'first', inplace = True)   
news = news.reset_index(drop='True')
news

EmptyDataError
EmptyDataError
EmptyDataError
EmptyDataError
EmptyDataError
EmptyDataError
ParserError
ParserError
EmptyDataError
EmptyDataError
EmptyDataError
EmptyDataError


,headlines,Date
0,Image copyright Jack TaylorCompanies looking f...,1.543338e+09
1,Image copyright Getty Images Image caption Ear...,1.543338e+09
2,Image copyright Getty ImagesThomas Cook has wa...,1.543338e+09
3,Image copyright Getty ImagesDoes being a found...,1.543338e+09
4,Image copyright Getty Images Image caption The...,1.543338e+09
5,Media playback is unsupported on your device M...,1.543338e+09
6,Media playback is unsupported on your device M...,1.543338e+09
7,Image copyright AFP Image caption Uber paid a ...,1.543338e+09
8,Image copyright Getty ImagesUS President Donal...,1.543338e+09
9,Image copyright Getty ImagesThe PM's Brexit de...,1.543338e+09


In [6]:
# Перевожу даты из timestamp в datetime
def num_to_time(num):
    return datetime.utcfromtimestamp(num)
news['Date'] = news['Date'].apply(num_to_time)
news = news.set_index('Date')
news = news.sort_index()

In [148]:
news.head()

,headlines
Date,
2018-11-27 16:44:46.514130,Washington (CNN Business) President Donald Tru...
2018-11-27 16:44:49.332132,"Washington (CNN) The US has ""slammed the brake..."
2018-11-27 16:44:50.457763,Moscow (CNN) Ukrainian lawmakers on Monday vot...
2018-11-27 16:44:51.485964,Hong Kong (CNN) The Chinese government has rev...
2018-11-27 16:44:52.300899,(CNN) Most politicians go to great lengths to ...


In [7]:
# разбиваю новости по часам
# рынок открывается в 10:00, новости за ночь не беру
# то есть новости между 10:00 и 11:00 присваиваю к 11:00, новости между 11:00 и 12:00 присваиваю к 12:00 и т.д.
# в плоть до 18:00
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
sp500_new = sp500.copy()
sp500_new['headlines_hourly'] = np.empty((len(sp500_new), 0)).tolist()
news_edit = news.copy()
night = datetime(2019, 1, 2, 10, 0).time()
eleven = datetime(2019, 1, 2, 11, 0).time()
from_date = 0
k = 0
start = datetime.now() 
for stock_date in sp500_new.index.tolist():

    if stock_date == sp500.index.tolist()[160]:
        print('20%')
        end = datetime.now() 
        print(end - start)   
    if stock_date == sp500.index.tolist()[240]:
        print('30%')
        end = datetime.now() 
        print(end - start)   
    if stock_date == sp500.index.tolist()[320]:
        print('40%')
        end = datetime.now() 
        print(end - start)   
    if stock_date == sp500.index.tolist()[400]:
        print('50%')
        end = datetime.now() 
        print(end - start)   
    if stock_date == sp500.index.tolist()[480]:
        print('60%')
        end = datetime.now() 
        print(end - start)   
    if stock_date == sp500.index.tolist()[560]:
        print('70%')
        end = datetime.now() 
        print(end - start)   
    if stock_date == sp500.index.tolist()[640]:
        print('80%')
        end = datetime.now() 
        print(end - start)   
    if stock_date == sp500.index.tolist()[720]:
        print('90%')
        end = datetime.now() 
        print(end - start)   
    if stock_date == sp500.index.tolist()[850]:
        print('99%')
    list_news = []
    first_news = -1
    last_news = -1
    if news_edit.index.tolist()[k] <= stock_date and k != len(news_edit.index.tolist()) and stock_date.time() != night and stock_date - news_edit.index.tolist()[k] <= timedelta(days = 0,hours = 1):

        first_news = k
        if from_date == 0:
            from_date = stock_date
    elif news_edit.index.tolist()[k] <= stock_date and k != len(news_edit.index.tolist()) and stock_date.time() == night:
        stock_date_temp = stock_date
    elif news_edit.index.tolist()[k] <= stock_date and k != len(news_edit.index.tolist()) and stock_date.time() == eleven:
        first_news = nearest(news_edit.index.tolist(),stock_date_temp)
        k = news_edit.index.tolist().index(first_news)
        if k+1 == len(news_edit.index.tolist()):
            break
        if first_news < stock_date_temp:
            first_news = k+1
        else:
            first_news = k
        if from_date == 0:
            from_date = stock_date

    #if news_edit.index.tolist()[k] <= stock_date and k != len(news_edit.index.tolist()) and stock_date == night and stock_date - news_edit.index.tolist()[k] <= timedelta(days = 0,hours = 1):
        
    if  k == len(news_edit.index.tolist()):
        to_date = stock_date
        
    if first_news != -1:
        last_news = nearest(news_edit.index.tolist(),stock_date)
        k = news_edit.index.tolist().index(last_news)
        if last_news > stock_date:
            k = news_edit.index.tolist().index(last_news)
            last_news = news_edit.index.tolist()[k - 1]
        else:
            k += 1
        #print('first - last')
        #print(news_edit.index.tolist()[first_news])
        #print(last_news)
        sp500_new['headlines_hourly'][stock_date].append(news_edit['headlines'][news_edit.index.tolist()[first_news]:last_news].tolist())
        to_date = stock_date
    if stock_date == sp500.index.tolist()[80]:
        print('10%')
        end = datetime.now() 
        print(end - start)   
        #break
        

        

    
end = datetime.now() 
print(end - start)     

# sp500_new = sp500_new[from_date:to_date]
# sp500_new = sp500_new[sp500_new.Content != '']
# sp500 = sp500_new.copy()    

10%
0:02:59.791898
20%
0:13:22.854005
30%
0:25:54.586157
40%
0:38:33.411828
50%
0:50:59.207786
60%
1:02:22.761199
70%
1:08:46.241468
80%
1:20:01.079794
90%
1:26:00.072352
99%
1:37:32.126729


In [9]:
# Удаляю часы в которых нет новостей
cop_sp500 = sp500_new.copy()

cop_sp500 = cop_sp500[cop_sp500.astype(str)['headlines_hourly'] != '[]']
cop_sp500 = cop_sp500[cop_sp500.astype(str)['headlines_hourly'] != '[[]]']
# перев
headlines = cop_sp500['headlines_hourly'].tolist()


In [10]:
cop_sp500

,Open,headlines_hourly
2018-11-27 17:00:00,-0.28,[[Washington (CNN Business) President Donald T...
2018-11-27 18:00:00,10.45,[[(CNN) A court in Russian-controlled Crimea h...
2018-11-28 11:00:00,-3.97,[[In asserting that investigators were unneces...
2018-11-28 12:00:00,6.24,[[JOHANNESBURG — A new study says many South A...
2018-11-28 13:00:00,28.08,[[Image copyright ReutersAll forms of Brexit w...
2018-11-28 14:00:00,8.85,[[GENEVA (Reuters) - Afghanistan President Ash...
2018-11-28 15:00:00,-4.81,"[[WASHINGTON, Nov 28 (Reuters) - President Don..."
2018-11-28 16:00:00,16.68,"[[SEHORE/BHOPAL, INDIA (Reuters) - Weak farm p..."
2018-11-28 17:00:00,-0.11,[[(CNN) Brain scans of men addicted to online ...
2018-11-28 18:00:00,-6.82,[[(CNN) Environmental Protection Agency chief ...


In [11]:
# Убираю лишнюю вложенность листа 
for i in range(len(headlines)):
    headlines[i] = headlines[i][0]
    

In [12]:
len(headlines)

493

In [13]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"its": "it is",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [152]:
# Функция приводит в нижнемиу регистру, заменяет сокращния на полные записи слов
# Удаляет не английские символы и знаки препинания
# Удаляет стоп слова
# Делаю Стемминг слов
from nltk.stem import PorterStemmer
ps = PorterStemmer()
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'[^a-z|" "]', '', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        text = [ps.stem(w) for w in text if not w in stops]
        text = " ".join(text)

    return text

In [153]:
price = cop_sp500['Open']


In [154]:
#Обрабатываю слова

clean_headlines = []
for daily_headlines in headlines:
    clean_daily_headlines = []
    for headline in daily_headlines:
        clean_daily_headlines.append(clean_text(headline))
    clean_headlines.append(clean_daily_headlines)

In [166]:
clean_headlines[0][0]

'court russiancontrol crimea order three ukrainian sailor captur confront russian forc sunday detain two month move like inflam tension two countriesa judg crimean town simferopol said tuesday vladimir varimez yuri budzylo vladimir tereshchenko arrest held januari accord russian state news agenc riath decis come confront two countri sunday around waterway link azov sea black sea russia fire seiz three ukrainian navi ship detain ukrainian sailorsukrain russia offer conflict account incid accus violat law seaukrain said two small gunboat one tugboat attack russian naval forc sunday enter kerch strait en rout citi mariupolread'

In [156]:
print(len(clean_headlines[0]))
print(len(clean_headlines[1]))
print(len(clean_headlines[2]))
print(len(clean_headlines[3]))

386
62
122
70


In [157]:

# Удаляю первый Час так как там около 350 новостей, когда в остальных часах 60-80 новостей, это странно
clean_headlines = clean_headlines[1:]
price = price[1:]

In [194]:
clean_joined = clean_headlines.copy()

In [195]:
# соединяю новости внури часа
# Так чтобы в одном часе получилась одна большая новость
for i in range(len(clean_headlines)):
    clean_joined[i] = ' '.join(clean_headlines[i])

In [159]:
# Перевожу дельты индексов в +1 и -1
price_new = np.empty((len(price)))
len(price_new)
for i in range(len(price)):
    if price[i] > 0 :
        price_new[i] = 1
    else:
        price_new[i] = -1
price_new

array([ 1., -1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,
        1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,
       -1.,  1.,  1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.,  1.,
        1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1., -1.,
       -1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1., -1.,  1.,
        1.,  1.,  1., -1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,  1.,
       -1., -1., -1.,  1., -1., -1., -1., -1.,  1.,  1., -1., -1., -1.,
       -1.,  1., -1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
       -1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,
        1., -1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1.,
        1.,  1., -1., -1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1., -1.,
        1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,
        1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1., -1.,  1.,  1.,
        1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1

In [202]:
# Split data into training and testing sets.
x_train, x_test, y_train, y_test = train_test_split(clean_joined, price_new, test_size = 0.20, random_state = 42)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [161]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
count_vect = CountVectorizer()
from sklearn.pipeline import Pipeline
from nltk.tokenize import sent_tokenize, word_tokenize

In [197]:
x_train[0]

'path pound hit month high euro follow increas specul delay brexitprim minist theresa may said common deal agre nodeal exit reject could short extens date britain leav euat one point sterl hit highest level sinc may howev mr may concess widerang investor hope caus sterl dip againpound v euroagainst us dollar reach one point highest level sinc end januari start lose groundpound v dollaranalyst jane foley rabobank said mr may remark common "discourag investors" give impress even brexit delay "the cliffedg could even sharper three month time""th market particularli like heard" saidhowev ad still consensu among investor nodeal brexit would avoid "neither parliament elector europ want it"th valu pound fell sharpli aftermath brexit referendum result tuesday bank england governor mark carney told mp treasuri committe bank would provid support economi event nodeal brexith said bank monetari polici committe emphasis respons shock nodeal brexit would depend econom situation"given except circumst

In [200]:
print(len(x_train))
print(len(y_test))

393
99


In [203]:
# Векторизую тренировочный сет новостей с помощью CountVectorizer
count_vect = CountVectorizer(ngram_range=(1, 2))
X_train_counts = count_vect.fit_transform(x_train)

# Подсчитываю частоту употребления слова с Tfidf
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

# Обучаю Наианый Бесовский классификатор
nb = MultinomialNB()
clf = nb.fit(X_train_tf, y_train)

# Векторизую.. тестовый сет новостей с помощью CountVectorizer
X_new_counts = count_vect.transform(x_test)
X_new_tfidf = tf_transformer.transform(X_new_counts)

# пердсказываю
predicted = clf.predict(X_new_tfidf)

#Вывожу точность
print(np.mean(predicted == y_test) )

0.5252525252525253


In [212]:
y_test

array([-1.,  1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,
        1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,
        1., -1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,  1.,  1., -1.,
        1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1.,
        1.,  1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,
       -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,
       -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,
        1., -1.,  1.,  1., -1.,  1.,  1.,  1.])

In [208]:
# Ложноположительный прогноз
predicted

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [206]:
F_names = count_vect.get_feature_names()
Vocab = dict(zip(F_names, nb.feature_count_[0]))
Vocab
import operator
sorted_x = sorted(Vocab.items(), key=operator.itemgetter(1),reverse=True)
sorted_x[:30]

[('said', 30.797872904977005),
 ('us', 12.6844396282951),
 ('year', 12.176700154272666),
 ('would', 11.724198632472836),
 ('percent', 9.63104084332124),
 ('say', 9.20409546907743),
 ('new', 9.196943478370903),
 ('also', 8.86020190782181),
 ('presid', 8.722795992649692),
 ('state', 8.454098077683998),
 ('govern', 8.180218771290022),
 ('peopl', 8.030616714130533),
 ('trump', 7.789044345625617),
 ('one', 7.7189069537528034),
 ('report', 7.359863213252099),
 ('compani', 7.139734256423545),
 ('may', 6.92262841410314),
 ('last', 6.851316710464536),
 ('two', 6.598429835381916),
 ('time', 6.438893755167901),
 ('could', 6.378735735976314),
 ('countri', 6.267208812234694),
 ('unit', 5.602877152160851),
 ('deal', 5.5243454802907745),
 ('includ', 5.338444768960629),
 ('told', 5.33003313074497),
 ('first', 5.167629719197018),
 ('like', 5.152487460382629),
 ('minist', 5.149756803106973),
 ('group', 5.037348267022605)]

In [207]:
Vocab = dict(zip(F_names, nb.feature_count_[1]))
Vocab
import operator
sorted_x = sorted(Vocab.items(), key=operator.itemgetter(1),reverse=True)
sorted_x[0:50]

[('said', 37.16020782439525),
 ('us', 14.531077508157471),
 ('year', 14.356788151713642),
 ('would', 13.974685650994711),
 ('percent', 11.425043061318476),
 ('new', 11.118431925687018),
 ('say', 11.055640028829814),
 ('also', 10.61088929282952),
 ('govern', 10.027708924758011),
 ('state', 9.965524810139641),
 ('presid', 9.903388721789783),
 ('peopl', 9.353077323198624),
 ('one', 9.017920150484443),
 ('compani', 8.831234383562565),
 ('report', 8.668276441044904),
 ('may', 8.634871982866578),
 ('trump', 8.420705812495406),
 ('last', 7.9855222443589655),
 ('two', 7.746490069194914),
 ('countri', 7.543043577615465),
 ('could', 7.515933641134255),
 ('time', 7.335824642925819),
 ('deal', 7.013412174742992),
 ('unit', 6.669726287269564),
 ('includ', 6.468375576238319),
 ('told', 6.173533667520747),
 ('like', 6.14968122742346),
 ('minist', 6.133773470596585),
 ('first', 6.065632876432419),
 ('group', 5.85679477242896),
 ('nation', 5.7996335513977355),
 ('day', 5.775121201906347),
 ('right', 5.

In [211]:
### SVM
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.linear_model import SGDClassifier


# parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
#                'tfidf__use_idf': (True, False),
#                'clf__alpha': (1e-2, 1e-3),}

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer(use_idf=True)),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5, random_state=42)),])
text_clf = text_clf.fit(x_train, y_train)
                     
# gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)                     
# gs_clf = gs_clf.fit(train_content, train_close)

predicted = text_clf.predict(x_test)
# predicted = gs_clf.predict(test_content)                    
np.mean(predicted == y_test) 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.47474747474747475

In [213]:
predicted

array([ 1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,
       -1.,  1., -1., -1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1.,
        1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.,
       -1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1.,
       -1., -1., -1., -1.,  1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,
       -1., -1.,  1., -1., -1., -1., -1., -1.])